## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="https://github.com/yatishkumarbn/agenticai_course/blob/main/assets/tools.png?raw=1" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs,
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 4.5 MB/s eta 0:00:00


In [4]:
!pip install gradio

In [3]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

#from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
reader = PdfReader("/content/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [5]:
print(linkedin)

   
Contact
9886173433 (Mobile)
yatishkumar.b.n@gmail.com
www.linkedin.com/in/yatish-kumar-
bn (LinkedIn)
Top Skills
Data-Driven Insights
Team Building
Revenue Management
Languages
English (Full Professional)
Telugu
Tamil
Hindi
Kannada
Certifications
Hortonworks Certified Associate
Microsoft Certified: Azure AI
Fundamentals
Fundamentals of MCP
Data Science and Bigdata Analytics
Associate (EMCDSA)
Microsoft Certified: Azure Data
Fundamentals
Honors-Awards
Associate Performance Award
Experian Excel Award
Experian Excel Award
Credit Excellence Award
Spot Award
Publications
ELECTRONIC VOTING SYSTEM
USING BLOCKCHAIN
Yatish Kumar
General Manager (OTT, Adtech, CLM) @ Airtel Digital | Data
Science
Bengaluru, Karnataka, India
Summary
Airtel Digital empowers businesses through innovative analytics and
data-driven solutions, with a focus on Adtech, OTT, and CLM. As
General Manager with over 7 months in this role and experience
in data architecture and analytics, contributed to advancing Airtel
Di

In [19]:
with open("/content/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [20]:
name = "Yatish"

In [21]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [22]:
system_prompt

"You are acting as Yatish. You are answering questions on Yatish's website, particularly questions related to Yatish's career, background, skills and experience. Your responsibility is to represent Yatish for interactions on the website as faithfully as possible. You are given a summary of Yatish's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Yatish. I'm Data Science and Analytics Leader. I'm originally from Bangalore, India having experience in UK for 3 years.\nI love all foods, particularly Spanish and Indian food. I'm not allergic, I love Hyderabad biriyani! I would love to mentor people looking for transition into Data career. I would love to eat jack fruit. I have a son who is 9 year old and his name is kiyansh. He going to Vidyashilpa school. I don't like APT or Blackpink songs.

In [23]:
from google.colab import userdata
import os

g_api_key = userdata.get('GOOGLE_API_KEY')

os.environ['GOOGLE_API_KEY'] = g_api_key
google_api_key = os.getenv('GOOGLE_API_KEY')

In [24]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

In [25]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=model_name, messages=messages)
    #response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [26]:
gr.ChatInterface(chat, type="messages").launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d93876b4155d652792.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [ ]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [ ]:
reply

In [ ]:
evaluate(reply, "do you hold a patent?", messages[:1])

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()